In [1]:
pip install imbalanced-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [15]:
from pyspark.sql import SparkSession
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col as spark_col
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
spark = SparkSession.builder \
    .appName("Insurance Fraud Detection") \
    .getOrCreate()


23/04/16 11:26:18 WARN Utils: Your hostname, DESKTOP-BBD74IS resolves to a loopback address: 127.0.1.1; using 172.23.40.194 instead (on interface eth0)
23/04/16 11:26:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/16 11:26:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
data_train = spark.read.csv("/home/jb-admin/data/df_train.csv", header=True, inferSchema=True)
data_test = spark.read.csv("/home/jb-admin/data/df_test.csv", header=True, inferSchema=True)

In [5]:
column_types = data_train.dtypes

# Print the types of all columns
for col, dtype in column_types:
    print(f"Column '{col}' has data type '{dtype}'")

Column 'months_as_customer' has data type 'double'
Column 'age' has data type 'double'
Column 'policy_deductable' has data type 'double'
Column 'umbrella_limit' has data type 'double'
Column 'incident_hour_of_the_day' has data type 'double'
Column 'number_of_vehicles_involved' has data type 'double'
Column 'bodily_injuries' has data type 'double'
Column 'witnesses' has data type 'double'
Column 'total_claim_amount' has data type 'double'
Column 'insured_sex_MALE' has data type 'double'
Column 'insured_education_level_College' has data type 'double'
Column 'insured_education_level_High School' has data type 'double'
Column 'insured_education_level_JD' has data type 'double'
Column 'insured_education_level_MD' has data type 'double'
Column 'insured_education_level_Masters' has data type 'double'
Column 'insured_education_level_PhD' has data type 'double'
Column 'insured_occupation_armed-forces' has data type 'double'
Column 'insured_occupation_craft-repair' has data type 'double'
Column 

In [52]:
data_train = data_train.withColumn("y_train", spark_col("y_train").cast("double"))
data_test = data_test.withColumn("y_test", spark_col("y_test").cast("double"))
data_train = data_train.withColumnRenamed("y_train", "y")
data_test = data_test.withColumnRenamed("y_test", "y")

In [7]:
data_train.head(5)

23/04/16 11:26:24 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[Row(months_as_customer=-0.917080506924668, age=-1.2046562179182658, policy_deductable=-0.22248156216849216, umbrella_limit=-0.46898138569178943, incident_hour_of_the_day=1.458062856610398, number_of_vehicles_involved=-0.7985511905478677, bodily_injuries=0.001730583030308226, witnesses=-1.355294174225357, total_claim_amount=-0.30254060189889886, insured_sex_MALE=1.0, insured_education_level_College=0.0, insured_education_level_High School=0.0, insured_education_level_JD=1.0, insured_education_level_MD=0.0, insured_education_level_Masters=0.0, insured_education_level_PhD=0.0, insured_occupation_armed-forces=0.0, insured_occupation_craft-repair=0.0, insured_occupation_exec-managerial=0.0, insured_occupation_farming-fishing=1.0, insured_occupation_handlers-cleaners=0.0, insured_occupation_machine-op-inspct=0.0, insured_occupation_other-service=0.0, insured_occupation_priv-house-serv=0.0, insured_occupation_prof-specialty=0.0, insured_occupation_protective-serv=0.0, insured_occupation_sale

In [53]:
categorical_columns = []
num_columns = [column for column in data_train.columns if column not in categorical_columns + ["y"]]
assembler = VectorAssembler(inputCols=num_columns, outputCol="features")
lr = LogisticRegression(featuresCol="features", labelCol="y")


In [54]:
stages = [assembler, lr]
pipeline = Pipeline(stages=stages)

In [10]:
data_train.show(5)


+-------------------+-------------------+--------------------+--------------------+------------------------+---------------------------+--------------------+--------------------+--------------------+----------------+-------------------------------+-----------------------------------+--------------------------+--------------------------+-------------------------------+---------------------------+-------------------------------+-------------------------------+----------------------------------+----------------------------------+------------------------------------+------------------------------------+--------------------------------+----------------------------------+---------------------------------+----------------------------------+------------------------+-------------------------------+-----------------------------------+------------------------+--------------------------------------+---------------------------+-----------------------------+-----------------------------+------------

In [55]:
fit_model = pipeline.fit(data_train)


In [56]:
results_test = fit_model.transform(data_test)
results_train = fit_model.transform(data_train)


In [57]:
# Make predictions on the test data
predictions = fit_model.transform(data_test)
# Instantiate a BinaryClassificationEvaluator object
evaluator = MulticlassClassificationEvaluator(labelCol="y")

# Calculate the AUC
auc = evaluator.evaluate(predictions)

print("AUC:", auc)

AUC: 0.7672372266682989


In [58]:
predictions = fit_model.transform(data_train)
# Instantiate a BinaryClassificationEvaluator object
evaluator = MulticlassClassificationEvaluator(labelCol="y")

# Calculate the AUC
auc = evaluator.evaluate(predictions)

print("AUC:", auc)

AUC: 0.858415435792754


KNN - not avaialbe in spark

In [61]:
## decision tree
dtc = DecisionTreeClassifier(featuresCol="features", labelCol="y")
data_train_assembled = assembler.transform(data_train)
data_test_assembled = assembler.transform(data_test)
model = dtc.fit(data_train_assembled)

predictions = model.transform(data_test_assembled)
evaluator = MulticlassClassificationEvaluator(labelCol="y")
test_acc = evaluator.evaluate(predictions)
                                              
# Compute the confusion matrix and classification report
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
mc_evaluator = MulticlassClassificationEvaluator(labelCol="y")
confusion_matrix = predictions.select("y", "prediction").rdd.map(lambda x: (x[1], x[0])).countByValue()
print("Confusion Matrix:")
print(confusion_matrix)

print("Classification Report:")
print(mc_evaluator.evaluate(predictions))

Confusion Matrix:
defaultdict(<class 'int'>, {(0.0, 0.0): 195, (1.0, 1.0): 29, (0.0, 1.0): 45, (1.0, 0.0): 31})
Classification Report:
0.7372382721585208


Adding hyper parameters

In [62]:
# Hyperparameter tuning
param_grid = ParamGridBuilder() \
    .addGrid(dtc.impurity, ["gini", "entropy"]) \
    .addGrid(dtc.maxDepth, [3, 5, 7, 10]) \
    .addGrid(dtc.minInstancesPerNode, range(2,10)) \
    .build()

cross_val = CrossValidator(estimator=dtc,
                           estimatorParamMaps=param_grid,
                           evaluator=evaluator,
                           numFolds=5)

cv_model = cross_val.fit(data_train_assembled)

# Evaluate
predictions = cv_model.transform(data_test_assembled)
auc = evaluator.evaluate(predictions)
print("Test AUC: {:.2f}%".format(100*auc))

Test AUC: 79.39%


In [64]:
# Create a RandomForestClassifier object and fit it to the training data
rfc = RandomForestClassifier(featuresCol="features", labelCol="y", numTrees=10)
model = rfc.fit(data_train_assembled)

# Make predictions on the test data and evaluate the model
predictions = model.transform(data_test_assembled)
evaluator = BinaryClassificationEvaluator(labelCol="y")
test_acc = evaluator.evaluate(predictions)
print("Testing accuracy of Random Forest Classifier is: {:.2f}%".format(100 * test_acc))

# Compute the confusion matrix and classification report
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
mc_evaluator = MulticlassClassificationEvaluator(labelCol="y")
confusion_matrix = predictions.select("y", "prediction").rdd.map(lambda x: (x[1], x[0])).countByValue()
print("Confusion Matrix:")
print(confusion_matrix)

print("Classification Report:")
print(mc_evaluator.evaluate(predictions))

# Hyperparameter Tuning
param_grid = ParamGridBuilder() \
    .addGrid(rfc.numTrees, [50, 70, 90, 120, 180, 200]) \
    .addGrid(rfc.maxDepth, [3, 5, 7, 10]) \
    .addGrid(rfc.maxBins, [16, 32, 64]) \
    .addGrid(rfc.impurity, ["gini", "entropy"]) \
    .addGrid(rfc.featureSubsetStrategy, ["sqrt", "log2"]) \
    .build()

cross_val = CrossValidator(estimator=rfc,
                           estimatorParamMaps=param_grid,
                           evaluator=evaluator,
                           numFolds=5)

cv_model = cross_val.fit(data_train_assembled)

# Evaluate
predictions = cv_model.transform(data_test_assembled)
test_acc = evaluator.evaluate(predictions)
print("Testing accuracy of Random Forest Classifier with hyperparameter tuning is: {:.2f}%".format(100 * test_acc))


Testing accuracy of Random Forest Classifier is: 73.65%
Confusion Matrix:
defaultdict(<class 'int'>, {(0.0, 0.0): 196, (1.0, 1.0): 38, (1.0, 0.0): 30, (0.0, 1.0): 36})
Classification Report:
0.7767931607109908
23/04/16 12:18:40 WARN DAGScheduler: Broadcasting large task binary with size 1009.8 KiB
23/04/16 12:19:07 WARN DAGScheduler: Broadcasting large task binary with size 1070.5 KiB
23/04/16 12:19:07 WARN DAGScheduler: Broadcasting large task binary with size 1280.5 KiB
23/04/16 12:19:08 WARN DAGScheduler: Broadcasting large task binary with size 1090.9 KiB
23/04/16 12:19:08 WARN DAGScheduler: Broadcasting large task binary with size 1313.9 KiB
23/04/16 12:19:09 WARN DAGScheduler: Broadcasting large task binary with size 1187.7 KiB
23/04/16 12:19:10 WARN DAGScheduler: Broadcasting large task binary with size 1203.8 KiB
23/04/16 12:19:11 WARN DAGScheduler: Broadcasting large task binary with size 1070.9 KiB
23/04/16 12:19:11 WARN DAGScheduler: Broadcasting large task binary with size 

In [65]:
mc_evaluator = MulticlassClassificationEvaluator(labelCol="y", metricName="weightedPrecision")
weighted_precision = mc_evaluator.evaluate(predictions)
print("Weighted Precision:", weighted_precision)

mc_evaluator = MulticlassClassificationEvaluator(labelCol="y", metricName="weightedRecall")
weighted_recall = mc_evaluator.evaluate(predictions)
print("Weighted Recall:", weighted_recall)

mc_evaluator = MulticlassClassificationEvaluator(labelCol="y", metricName="f1")
f1_score = mc_evaluator.evaluate(predictions)
print("F1-score:", f1_score)

mc_evaluator = MulticlassClassificationEvaluator(labelCol="y", metricName="precisionByLabel")
precision_by_label = mc_evaluator.evaluate(predictions)
print("Precision by Label:", precision_by_label)

mc_evaluator = MulticlassClassificationEvaluator(labelCol="y", metricName="recallByLabel")
recall_by_label = mc_evaluator.evaluate(predictions)
print("Recall by Label:", recall_by_label)

mc_evaluator = MulticlassClassificationEvaluator(labelCol="y", metricName="f1MeasureByLabel")
f1_score_by_label = mc_evaluator.evaluate(predictions

SyntaxError: incomplete input (3104873531.py, line 22)